# Svensson (1994)



$$ r(t) = \beta_1 + \beta_2\frac{1-e^{-\lambda_1 t}}{\lambda_1 t} 
        + \beta_3 \left(\frac{1-e^{-\lambda_1 t}}{\lambda_1 t}-e^{-\lambda_1 t}\right)
        + \beta_4 \left(\frac{1-e^{-\lambda_2 t}}{\lambda_2 t}-e^{-\lambda_2 t}\right)$$

### Importing the data

In [473]:
from scipy.optimize import fmin
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from workadays import workdays as wd

path = r"C:\Users\Alysson\Documents\GitHub\Monetary-Shocks\Brasil\ETTJ\Base_LTN.xlsx"
path2 = r"C:\Users\Alysson\Documents\GitHub\Monetary-Shocks\Brasil\ETTJ\Parametros.xlsx"
df = pd.read_excel(path)
parameters = pd.read_excel(path2)

In [474]:
parameters.set_index("Data referência", inplace=True)
df["Maturity"] = df.apply(lambda row: wd.networkdays(row["DATA_REFERENCIA"], row["DATA_VENCIMENTO"]), axis=1)/252
df.set_index("DATA_REFERENCIA", inplace=True)

### Generating individuals

In [475]:
N =1200    ### Number of individuals 
p = int(2/3*N)      ## Proportion of individuals generated by method 1
var = 0.5

### 1 - Initial Values of the results of the estimation of the previous day

ref_date_str = "2023-07-27"
ref_date = datetime.strptime(ref_date_str, "%Y-%m-%d")
previous_date = ref_date - timedelta(days=1)
previous_parameters = parameters.loc[previous_date][1:]
beta_star_1 = np.array([previous_parameters.replace(previous_parameters[1],previous_parameters[0])])
e_previous = beta_star_1.T*np.random.normal(0, var, size=(6, p))
beta_previous = beta_star_1.T + e_previous

### 2 - Approximation that takes into account the observed yield to maturity (ytm)

data_estimada = df.loc[ref_date_str]
data_estimada.sort_values("Maturity")
b1 = data_estimada["EXPECTATIVA"][0]/100
b2 = data_estimada["EXPECTATIVA"][-1]/100 - data_estimada["EXPECTATIVA"][0]/100
b3 = 0
b4 = 0
lbda1 = (data_estimada["Maturity"][-1]-data_estimada["Maturity"][0])/2
lbda2 = lbda1
beta_star_2 = np.array([b1, b2, b3 , b4, lbda1, lbda2])[:, np.newaxis]
e_approx = beta_star_2*np.random.normal(0, var, size=(6, N-p))
beta_approx = beta_star_2 + e_approx ## ajustar colunas

### Adding constraints to the parameters

def update (betas,beta_star):
    updated_value = []
    for index, parameter in enumerate(betas):
        if parameter[0]<0:
            betas[index][0] = beta_star[0]+ beta_star[0]*np.random.normal(0, var)
            updated_value.append(betas[index][0]<0)
        if (parameter[0]+parameter[1])<0:
            betas[index][1] = beta_star[1]+ beta_star[1]*np.random.normal(0, var)
            updated_value.append((betas[index][0]+betas[index][1])<0)
        if parameter[4]<0:
            betas[index][4] = beta_star[4]+ beta_star[4]*np.random.normal(0, var)
            updated_value.append(betas[index][4]<0)
        if parameter[5]<0:
            betas[index][5] = beta_star[5]+ beta_star[5]*np.random.normal(0, var)
            updated_value.append(betas[index][5]<0)
    if sum(updated_value) != 0:
        update(betas,beta_star)
    return(beta_previous.T)

update(beta_previous.T,beta_star_1.T)
update(beta_approx,beta_star_2)

individuals = np.concatenate((beta_previous.T, beta_approx.T), axis=0)

In [476]:
individuals

array([[ 0.12245414,  0.08456037,  0.05670799, -0.06831825,  2.5626974 ,
         1.1686942 ],
       [ 0.05618134,  0.02461252,  0.09846695, -0.0058412 ,  0.95586825,
         1.50752948],
       [ 0.07542064,  0.08130873,  0.08704944, -0.09709566,  1.23701501,
         1.60563815],
       ...,
       [ 0.05717559,  0.0035304 ,  0.        ,  0.        ,  1.57733784,
         2.78176265],
       [ 0.13099944, -0.03923089,  0.        ,  0.        ,  2.78904439,
         1.97606133],
       [ 0.11925914, -0.03832196,  0.        ,  0.        ,  0.90056328,
         0.64977586]])

### Selection

In [477]:
### add duration and coupons
values = []

### 40% of the individuals survive
s = int(0.4 * N)

for row in individuals:
    def myval(c):
        df['NSS'] =(c[0])+(c[1]*((1-np.exp(-df['Maturity']*c[4]))/(df['Maturity']*c[4])))+(c[2]*((((1-np.exp(-df['Maturity']*c[4]))/(df['Maturity']*c[4])))-(np.exp(-df['Maturity']*c[4]))))+(c[3]*((((1-np.exp(-df['Maturity']*c[5]))/(df['Maturity']*c[5])))-(np.exp(-df['Maturity']*c[5]))))
        df['Calculated_price'] = 1000 / (1 + df['NSS']) ** df['Maturity'] 
        df['Residual'] =  (df['PU'] - df['Calculated_price'])**2
        val = np.sum(df['Residual'])
        return(val)
    
    val = myval(row)
    values.append((val, row)) 
sol = pd.DataFrame(values, columns=['Calculated Value', "Parameters"])

selection = sol.sort_values('Calculated Value')[0:s]

In [478]:
selection.head()

,Calculated Value,Parameters
233,95.537734,"[0.10854927844430576, 0.10035677608002838, 0.0..."
499,184.079297,"[0.12083822327244219, 0.03352891454079132, 0.0..."
95,186.034487,"[0.10473210389203097, 0.02042628025230457, 0.0..."
561,273.105906,"[0.0821922659631091, 0.06339182590387227, 0.05..."
398,280.246647,"[0.10258078955663345, 0.12386932610737232, -0...."


### Cross-Over

In [479]:
alpha = 6
next_gen = []


for num in range(N):
    psi = np.random.uniform(0, 1)
    theta_r = np.concatenate(selection.iloc[(np.random.beta(1,alpha, 1)*s),:]["Parameters"].values)
    theta_s = np.concatenate(selection.iloc[(np.random.beta(1,alpha, 1)*s),:]["Parameters"].values)
    next_gen.append(psi*theta_r+(1-psi)*theta_s)



### Mutation

In [485]:
pi = 0.35
var = 0.5 ## ajustar a var para a geracao maior que 2
e_mutation = []

for num in range(N):
    e = np.random.normal(0, var, 6)*np.random.choice([0, 1], size=6, p=[1 - pi, pi])
    e_mutation.append(e)

new_individuals = np.array(next_gen) + np.array(e_mutation)




In [481]:
new_individuals

array([[-0.96153091, -0.02358666,  0.2913874 ,  0.        ,  2.00273583,
         0.82494584],
       [ 0.56411326,  0.44415244, -0.01757662, -0.09562009,  2.66876719,
         1.43593193],
       [ 0.12020355, -1.01153933,  0.        ,  0.        ,  0.91317106,
         1.77413258],
       ...,
       [-0.13015245,  0.02954039,  0.16455456, -1.03615084,  4.09089702,
         1.66443286],
       [ 0.09604241,  0.1010123 ,  0.71456218,  0.4968316 ,  2.79498682,
         2.78870965],
       [ 0.11184791, -0.26570323,  0.072517  , -0.12959884,  3.02058707,
         1.08455048]])

In [482]:




for row in new_individuals:
    def myval(c):
        df['NSS'] =(c[0])+(c[1]*((1-np.exp(-df['Maturity']*c[4]))/(df['Maturity']*c[4])))+(c[2]*((((1-np.exp(-df['Maturity']*c[4]))/(df['Maturity']*c[4])))-(np.exp(-df['Maturity']*c[4]))))+(c[3]*((((1-np.exp(-df['Maturity']*c[5]))/(df['Maturity']*c[5])))-(np.exp(-df['Maturity']*c[5]))))
        df['Calculated_price'] = 1000 / (1 + df['NSS']) ** df['Maturity'] 
        df['Residual'] =  (df['PU'] - df['Calculated_price'])**2
        val = np.sum(df['Residual'])
        return(val)
    
    val = myval(row)
    values.append((val, row)) 
sol = pd.DataFrame(values, columns=['Calculated Value', "Parameters"])

selection = sol.sort_values('Calculated Value')[0:s]

In [486]:
selection

,Calculated Value,Parameters
1986,0.000000,"[-1.2510393856549902, 0.06590649024707723, 0.0..."
2103,0.000000,"[-1.1503165090960845, 0.008218500893837858, 0...."
1677,0.000000,"[-1.1268698655042184, 0.09727319684684269, -0...."
1685,63.094458,"[0.10696351793341444, 0.016659328666372566, 0...."
2331,91.225420,"[0.10619637283606155, 0.08169556606032694, 0.0..."
...,...,...
923,5741.498962,"[0.1305765996341619, -0.05784953718283861, 0.0..."
1593,5749.970141,"[0.10117460507178092, -0.11565800299105694, 0...."
1357,5819.415760,"[0.1438803477733652, -0.03235576506295223, 0.0..."
315,5826.844186,"[0.08787564356716038, 0.1134087332983512, 0.10..."
